## Modelo de regresión logística para identificar posibles clientes

### 1. Carga de paquetes y data

In [2]:
#Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile as zf

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE

%matplotlib inline

In [3]:
#Cargar el .zip
datazip=zf.ZipFile('Bank_Campaign.csv.zip')
#Archivos en el .zip
datazip.namelist()
#Lectura de los datos
data=pd.read_csv(datazip.open('Bank_Campaign.csv'),';')

C:\Users\willi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3397: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
